# Machine Learning Project: Classification 3
Calvin Wong<br>
Mohammed Hussain

In [1]:
# Useful libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVC

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, classification_report
from sklearn import metrics  
from sklearn import tree

# Classification

In [2]:
Test = pd.read_csv("TestData3.csv", header=None)
Test

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,1,5,3,4,1,5,1,3,0,1,1,7,1
1,2,1,4,3,5,5,3,2,0,1,1,7,1
2,1,5,2,3,9,2,1,4,1,3,1,7,1
3,1,2,2,2,3,2,1,4,2,3,3,7,1
4,2,5,3,4,1,2,1,1,0,2,2,7,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2688,2,5,1,1,2,5,1,3,2,3,1,7,1
2689,1,5,2,4,1,5,1,4,0,3,1,7,1
2690,2,5,1,2,1,5,1,3,2,3,1,7,1
2691,1,1,6,4,3,5,2,3,1,2,3,7,1


In [3]:
Train = pd.read_csv("TrainData3.csv", header=None)
Train

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,2,1.0,5,4.0,5.0,5.000000e+00,3,3.0,0,1.0,1.0,7.0,1.000000e+99
1,1,1.0,5,5.0,5.0,5.000000e+00,3,5.0,2,1.0,1.0,7.0,1.000000e+00
2,2,1.0,3,5.0,1.0,5.000000e+00,2,3.0,1,2.0,3.0,7.0,1.000000e+00
3,2,5.0,1,2.0,6.0,5.000000e+00,1,4.0,2,3.0,1.0,7.0,1.000000e+00
4,2,5.0,1,2.0,6.0,3.000000e+00,1,4.0,2,3.0,1.0,7.0,1.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6295,1,5.0,4,5.0,3.0,1.000000e+99,1,1.0,0,2.0,3.0,7.0,1.000000e+00
6296,2,1.0,3,3.0,9.0,1.000000e+99,3,4.0,2,1.0,1.0,8.0,1.000000e+00
6297,2,1.0,4,4.0,6.0,5.000000e+00,3,4.0,2,1.0,1.0,7.0,1.000000e+00
6298,1,3.0,3,4.0,1.0,5.000000e+00,1,1.0,0,1.0,1.0,7.0,1.000000e+00


In [4]:
Label = pd.read_csv("TrainLabel3.csv", header=None)
Label

,0
0,9
1,9
2,9
3,1
4,1
...,...
6295,7
6296,9
6297,5
6298,9


In [5]:
df = pd.concat([Label,Train], axis=1, ignore_index=True)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,9,2,1.0,5,4.0,5.0,5.000000e+00,3,3.0,0,1.0,1.0,7.0,1.000000e+99
1,9,1,1.0,5,5.0,5.0,5.000000e+00,3,5.0,2,1.0,1.0,7.0,1.000000e+00
2,9,2,1.0,3,5.0,1.0,5.000000e+00,2,3.0,1,2.0,3.0,7.0,1.000000e+00
3,1,2,5.0,1,2.0,6.0,5.000000e+00,1,4.0,2,3.0,1.0,7.0,1.000000e+00
4,1,2,5.0,1,2.0,6.0,3.000000e+00,1,4.0,2,3.0,1.0,7.0,1.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6295,7,1,5.0,4,5.0,3.0,1.000000e+99,1,1.0,0,2.0,3.0,7.0,1.000000e+00
6296,9,2,1.0,3,3.0,9.0,1.000000e+99,3,4.0,2,1.0,1.0,8.0,1.000000e+00
6297,5,2,1.0,4,4.0,6.0,5.000000e+00,3,4.0,2,1.0,1.0,7.0,1.000000e+00
6298,9,1,3.0,3,4.0,1.0,5.000000e+00,1,1.0,0,1.0,1.0,7.0,1.000000e+00


# Input any missing data

In [6]:
# Separate numerical columns
numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns


# Impute missing values for numerical columns using the median
for col in numerical_cols:
    if df[col].isnull().sum() > 0:
        median_value = df[col].median()
        df[col] = df[col].fillna(median_value)
        print(f"Imputed missing values in '{col}' with median: {median_value}")
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,9,2,1.0,5,4.0,5.0,5.000000e+00,3,3.0,0,1.0,1.0,7.0,1.000000e+99
1,9,1,1.0,5,5.0,5.0,5.000000e+00,3,5.0,2,1.0,1.0,7.0,1.000000e+00
2,9,2,1.0,3,5.0,1.0,5.000000e+00,2,3.0,1,2.0,3.0,7.0,1.000000e+00
3,1,2,5.0,1,2.0,6.0,5.000000e+00,1,4.0,2,3.0,1.0,7.0,1.000000e+00
4,1,2,5.0,1,2.0,6.0,3.000000e+00,1,4.0,2,3.0,1.0,7.0,1.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6295,7,1,5.0,4,5.0,3.0,1.000000e+99,1,1.0,0,2.0,3.0,7.0,1.000000e+00
6296,9,2,1.0,3,3.0,9.0,1.000000e+99,3,4.0,2,1.0,1.0,8.0,1.000000e+00
6297,5,2,1.0,4,4.0,6.0,5.000000e+00,3,4.0,2,1.0,1.0,7.0,1.000000e+00
6298,9,1,3.0,3,4.0,1.0,5.000000e+00,1,1.0,0,1.0,1.0,7.0,1.000000e+00


# SVM Model

In [7]:
# Split into features (X) and target variable (y)
X = df.drop(df.columns[0], axis=1)
y = df[df.columns[0]]

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create an SVM model with a linear kernel
svm_model = SVC(kernel='linear', random_state=42)

# Train the model
svm_model.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = svm_model.predict(X_test)

# Evaluate the model's accuracy
f1 = f1_score(y_test, y_pred,average='weighted', zero_division= np.nan)
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred,average='weighted')
precision = precision_score(y_test, y_pred,average='weighted', zero_division= np.nan)

print("Accuracy:", accuracy)
print("F1 Score:", f1)
print("Recall:", recall)
print("Precision:", precision)

Accuracy: 0.14285714285714285
F1 Score: 0.12192999192885932
Recall: 0.14285714285714285
Precision: 0.20188541755401682


# KNN Model

In [8]:
# Split into features (X) and target variable (y)
X = df.drop(df.columns[0], axis=1)
y = df[df.columns[0]]

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


# Initialize the KNN classifier with k=5
knn_model = KNeighborsClassifier(n_neighbors=5)

# Train the model
knn_model.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = knn_model.predict(X_test_scaled)

# Evaluate the model's accuracy
f1 = f1_score(y_test, y_pred,average='weighted', zero_division= np.nan)
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred,average='weighted')
precision = precision_score(y_test, y_pred,average='weighted', zero_division= np.nan)

print("Accuracy:", accuracy)
print("F1 Score:", f1)
print("Recall:", recall)
print("Precision:", precision)


Accuracy: 0.24444444444444444
F1 Score: 0.23030123534414274
Recall: 0.24444444444444444
Precision: 0.22558699315403885
